In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data from PostgreSQL
query = 'SELECT * FROM "overall_policy_level_data_EF";'
data = pd.read_sql(query, con=engine)

# Step 2: Split data based on 'Zone 2' and save to CSV
zones = ['South', 'North', 'East', 'West']

try:
    for zone in zones:
        # Filter data for the current zone
        zone_data = data[data['Zone 2'].str.lower() == zone.lower()]
        
        # Define the file name
        file_name = f"Actual_Data_{zone}.csv"
        
        # Save the filtered data to a CSV file
        zone_data.to_csv(file_name, index=False)
        print(f"Data for {zone} saved to {file_name}")
except Exception as e:
    print("An error occurred:", e)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123', 
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data from PostgreSQL
query = 'SELECT * FROM "overall_policy_level_data_EF";'
data = pd.read_sql(query, con=engine)

# Step 2: Implement logic to update 'Zone 2' for Type B where it is NULL
# Merge data on 'Policy No' to bring Type A's 'Zone 2' into Type B rows
merged_data = data.merge(
    data[data['Type'] == 'A'][['Policy No', 'Zone 2']],
    on='Policy No',
    how='left',
    suffixes=('', '_A')
)

# Update 'Zone 2' in Type B rows where it is NULL
merged_data['Zone 2'] = merged_data.apply(
    lambda row: row['Zone 2_A'] if pd.isnull(row['Zone 2']) and row['Type'] == 'B' else row['Zone 2'], axis=1
)

# Drop the extra 'Zone 2_A' column used for the logic
merged_data = merged_data.drop(columns=['Zone 2_A'])

# Step 3: Save the updated data back to the database or a CSV file
merged_data.to_sql('overall_policy_level_data_EF_updated', con=engine, if_exists='replace', index=False)
print("Updated data saved to the database.")

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Step 1: Load data from PostgreSQL
query = 'SELECT * FROM public."corrected_merged_claim_data_EF";'
data = pd.read_sql(query, con=engine)

# Step 2: Split data based on 'Zone 2' and save to CSV
zones = ['South', 'North', 'East', 'West']

try:
    for zone in zones:
        # Filter data for the current zone
        zone_data = data[data['Zone 2'].str.lower() == zone.lower()]
        
        # Define the file name
        file_name = f"Actual_Base_PR_Data_{zone}.csv"
        
        # Save the filtered data to a CSV file
        zone_data.to_csv(file_name, index=False)
        print(f"Data for {zone} saved to {file_name}")
except Exception as e:
    print("An error occurred:", e)

Data for South saved to Actual_Base_PR_Data_South.csv
Data for North saved to Actual_Base_PR_Data_North.csv
Data for East saved to Actual_Base_PR_Data_East.csv
Data for West saved to Actual_Base_PR_Data_West.csv
